
| EDA dataframe      | Description |due|
| ----------- | ----------- |----------- |
| tomato   |토마토 가격/거래량 + 부산 날씨   +  소비자 물가지수 + 유가     |20190101 ~ 20200928|
| grape   | 캠벨얼리 가격/거래량 + 남원 날씨   +  소비자 물가지수 + 유가    |20190101 ~ 20200928|
| shinemuscat   |샤인마스캇  가격/거래량 + 구미 날씨   +  소비자 물가지수 + 유가          |20190101 ~ 20200928|
| train   |   tomato +  grape +  shinemuscat   | 20190101 ~ 20200928|
| test   |   tomato +  grape +  shinemuscat   | 20200929 ~ 20200928|
| df      | train + test       | 20190101 ~ 20201105|

# 외부데이터 수집

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'
# import tensorflow as tf
# import keras
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# from tensorflow.keras import Sequential
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense, LSTM, Dropout
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from statsmodels.tsa.seasonal import seasonal_decompose, STL
import urllib.request
import requests
import json
import datetime
import os
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/var/folders/lm/z4n_2b8s6rj3_vv57sg789kc0000gn/T/ipykernel_15640/238796418.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# 데이터셋(dataset)

**train** 20160101 ~ 20200928 <br> 
**test** 20200929 ~ 20201105

EDA와 기본 모델을 만들때 사용 <br>
train + test 합친 프레임(토마토, 포도) = df

In [4]:
raw_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/'
processed_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/processed/'

grape_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/grape/'
shinemuscat_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/shinemuscat/'

In [5]:
# test 데이터 합치기
arr = os.listdir('/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/data/public_data/test_files')
print(arr)
print()

filename_list = arr
filename_list_2020 = []    
for filename in filename_list :
    if '2020' in filename :
        filename_list_2020.append(filename)
        
path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/data/public_data/test_files/'
df_list = []
for filename in tqdm(filename_list_2020) :
    df = pd.read_csv(path + filename)
    df_list.append(df)
test = pd.concat(df_list, sort=False).reset_index(drop=True)

['test_2020-10-12.csv', 'test_2020-10-06.csv', 'test_2020-10-07.csv', 'test_2020-10-13.csv', 'test_2020-09-30.csv', 'test_2020-10-05.csv', 'test_2020-10-11.csv', 'test_2020-10-10.csv', 'test_2020-10-04.csv', 'test_2020-10-14.csv', 'test_2020-10-28.csv', 'test_2020-10-29.csv', 'test_2020-10-15.csv', 'test_2020-10-01.csv', 'test_2020-10-17.csv', 'test_2020-10-03.csv', 'test_2020-10-02.csv', 'test_2020-10-16.csv', 'test_2020-10-27.csv', 'test_2020-10-26.csv', 'test_2020-10-18.csv', 'test_2020-10-24.csv', 'test_2020-10-30.csv', 'test_2020-10-31.csv', 'test_2020-10-25.csv', 'test_2020-10-19.csv', 'test_2020-10-21.csv', 'test_2020-10-09.csv', 'test_2020-10-08.csv', 'test_2020-10-20.csv', 'test_2020-09-29.csv', 'test_2020-10-22.csv', 'test_2020-10-23.csv', '.ipynb_checkpoints', 'test_2020-11-04.csv', 'test_2020-11-05.csv', 'test_2020-11-02.csv', 'test_2020-11-03.csv', 'test_2020-11-01.csv']



100%|████████████████████████████████████████████████████| 38/38 [00:00<00:00, 777.44it/s]


In [459]:
# grape = pd.read_csv(processed_path+'grape.csv')
# shinemuscat = pd.read_csv(processed_path+'shinemuscat.csv')

train = pd.read_csv(raw_path+'train.csv')
test = pd.read_csv(raw_path+'test.csv')
df = pd.concat([train, test], axis = 0)
df = df.reset_index(drop=True)

print(train.shape)
print(test.shape)
print(df.shape)

(1733, 44)
(37, 44)
(1770, 44)


In [752]:
grape = df[['date', '요일', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)']]
shinemuscat = df[['date', '요일','샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)']]

print(grape.shape)
print(shinemuscat.shape)

(1770, 4)
(1770, 4)


In [602]:
arr = os.listdir('/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/')
print(arr)
print()
grape_arr2 = os.listdir('/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/grape/')
print(grape_arr2)
print()
shinemuscat_arr2 = os.listdir('/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/shinemuscat//')
print(shinemuscat_arr2)

['private_data.csv', '.DS_Store', 'test.csv', 'README.md', '최저시급.csv', 'shinemuscat', '물가지수.csv', '주유소_평균판매가격(201105).csv', 'train.csv', 'grape', '실업률_경제활동인구.csv', 'sample_submission.csv']

['금산_날씨.csv', '.DS_Store', '남원_날씨.csv', '수원_날씨.csv', '포도_수출입_201601020201130.csv', '거창_날씨.csv', '재배면적_생산량.csv', '구미_날씨.csv']

['금산_날씨.csv', '.DS_Store', '영천_날씨.csv', '포도_수출입_201601020201130.csv', '샤인머스켓_블루베리.csv', '재배면적_생산량.csv', '구미_날씨.csv']


# 캠벨얼리 외부데이터 불러오기 및 전처리

**날씨 데이터 지역 비율 >>> <br>
구미_날씨(김천) = 31.7%, 남원_날씨 = 28%, 수원_날씨(화성) = 20.5%, 금산_날씨(영동) = 14.7%, 거창_날씨 = 5.1%**

거래량, 가격, 자동차용경유, 소비자 물가 지수, 날씨(4개-평균기온, 평균온량, 일강수량, 일교차), 수출입(4개), 재배면적, 샤인머스켓(빅카인즈) 거래량(워드클라우드)   , 실업률_경제활동인구, 최저시급

### 날씨(weather, weather2)

In [718]:
### 날씨 데이터 가져오기 및 전처리  => 남원_날씨, 영천_날씨, 수원_날씨, 거창_날씨, 구미_날씨
gumi = pd.read_csv(grape_path+'구미_날씨.csv', index_col=0)
namwon = pd.read_csv(grape_path+'남원_날씨.csv', index_col=0)
suwon = pd.read_csv(grape_path+'수원_날씨.csv', index_col=0)
geumsan = pd.read_csv(grape_path+'금산_날씨.csv', index_col=0)
geochang = pd.read_csv(grape_path+'거창_날씨.csv', index_col=0)

In [719]:
# 상관계수가 높아서 drop -> 평균기온 & 일교차 대체 사용

gumi.drop(columns=['최고기온', '최저기온'],inplace=True)
namwon.drop(columns=['최고기온', '최저기온'],inplace=True)
suwon.drop(columns=['최고기온', '최저기온'],inplace=True)
geumsan.drop(columns=['최고기온', '최저기온'],inplace=True)
geochang.drop(columns=['최고기온', '최저기온'],inplace=True)

In [720]:
# 컬럼 이름 바꿈

gumi.rename(columns={'평균기온':'구미_평균기온','평균운량':'구미_평균운량','일강수량':'구미_일강수량','일교차':'구미_일교차'}, inplace=True)
namwon.rename(columns={'평균기온':'남원_평균기온','평균운량':'남원_평균운량','일강수량':'남원_일강수량','일교차':'남원_일교차'}, inplace=True)
suwon.rename(columns={'평균기온':'수원_평균기온','평균운량':'수원_평균운량','일강수량':'수원_일강수량','일교차':'수원_일교차'}, inplace=True)
geumsan.rename(columns={'평균기온':'금산_평균기온','평균운량':'금산_평균운량','일강수량':'금산_일강수량','일교차':'금산_일교차'}, inplace=True)
geochang.rename(columns={'평균기온':'거창_평균기온','평균운량':'거창_평균운량','일강수량':'거창_일강수량','일교차':'거창_일교차'}, inplace=True)

In [721]:
weather = pd.merge(gumi, namwon)
weather = pd.merge(weather, suwon)
weather = pd.merge(weather, geumsan)
weather = pd.merge(weather, geochang)

print(weather.shape)
weather.head(2)

(1771, 21)


,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,수원_일강수량,수원_일교차,금산_평균기온,금산_평균운량,금산_일강수량,금산_일교차,거창_평균기온,거창_평균운량,거창_일강수량,거창_일교차
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,0.0,9.9,-0.7,0.0,0.0,11.9,-1.1,0.0,0.0,16.5
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,0.0,9.1,4.8,0.0,0.0,13.8,1.4,0.0,0.0,18.3


In [722]:
weather.columns

Index(['날짜', '구미_평균기온', '구미_평균운량', '구미_일강수량', '구미_일교차', '남원_평균기온', '남원_평균운량',
       '남원_일강수량', '남원_일교차', '수원_평균기온', '수원_평균운량', '수원_일강수량', '수원_일교차',
       '금산_평균기온', '금산_평균운량', '금산_일강수량', '금산_일교차', '거창_평균기온', '거창_평균운량',
       '거창_일강수량', '거창_일교차'],
      dtype='object')

In [723]:
# 구미_날씨(김천) = 31.7%, 남원_날씨 = 28%, 수원_날씨(화성) = 20.5%, 금산_날씨(영동) = 14.7%, 거창_날씨 = 5.1%

weather2 = weather.copy()

weather2['재배지역_평균기온'] = weather2['구미_평균기온'] * (31.7/100) + weather2['남원_평균기온'] * (28/100) + weather2['수원_평균기온'] * (20.5/100)+ weather2['금산_평균기온'] * (14.7/100)+ weather2['거창_평균기온'] * (5.1/100)
weather2['재배지역_평균운량'] = weather2['구미_평균운량'] * (31.7/100) + weather2['남원_평균운량'] * (28/100) + weather2['수원_평균운량'] * (20.5/100)+ weather2['금산_평균운량'] * (14.7/100)+ weather2['거창_평균운량'] * (5.1/100)
weather2['재배지역_일강수량'] = weather2['구미_일강수량'] * (31.7/100) + weather2['남원_일강수량'] * (28/100) + weather2['수원_일강수량'] * (20.5/100)+ weather2['금산_일강수량'] * (14.7/100)+ weather2['거창_일강수량'] * (5.1/100)
weather2['재배지역_일교차'] = weather2['구미_일교차'] * (31.7/100) + weather2['남원_일교차'] * (28/100) + weather2['수원_일교차'] * (20.5/100)+ weather2['금산_일교차'] * (14.7/100)+ weather2['거창_일교차'] * (5.1/100)

weather2.head(2)

,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,금산_일강수량,금산_일교차,거창_평균기온,거창_평균운량,거창_일강수량,거창_일교차,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,0.0,11.9,-1.1,0.0,0.0,16.5,0.2958,1.107,0.0,12.0453
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,0.0,13.8,1.4,0.0,0.0,18.3,3.6117,1.312,0.0,13.1146


In [724]:
# 날짜 칼럼 추가
date = df['date']
date.loc[1770] = '2020-11-05'

In [725]:
weather2['date'] = date
weather2.head(2)

,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,금산_일교차,거창_평균기온,거창_평균운량,거창_일강수량,거창_일교차,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,date
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,11.9,-1.1,0.0,0.0,16.5,0.2958,1.107,0.0,12.0453,2016-01-01
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,13.8,1.4,0.0,0.0,18.3,3.6117,1.312,0.0,13.1146,2016-01-02


In [726]:
weather2 = weather2[['date','재배지역_평균기온', '재배지역_평균운량', '재배지역_일강수량', '재배지역_일교차']]
weather2 = weather2.round(2)

print(weather2.shape)
display(weather2.head(2))
display(weather2.tail(2))

(1771, 5)


,date,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
0,2016-01-01,0.30,1.11,0.0,12.05
1,2016-01-02,3.61,1.31,0.0,13.11


,date,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
1769,2020-11-04,4.65,0.22,0.0,12.64
1770,2020-11-05,7.47,4.71,0.0,18.14


### 소비자물가지수 & 오일(공통-포도)

In [80]:
product = pd.read_csv(raw_path+'물가지수.csv', index_col=0)
product.reset_index(drop=True, inplace=True)
product.rename(columns={'지수' : '소비자물가지수'}, inplace=True)

print(product.shape)

(1771, 1)


In [81]:
oil = pd.read_csv(raw_path+'주유소_평균판매가격(201105).csv',encoding='cp949')
oil.drop([1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780], inplace=True)

print(oil.shape)

(1771, 5)


### 수출입 데이터 전처리(월단위, 평균) (공통-포도)

In [121]:
grape_export = pd.read_csv(grape_path+'포도_수출입_201601020201130.csv', encoding='cp949')
grape_export.shape

(63, 6)

In [122]:
grape_export.rename(columns={'2016':'년','합':'월','1,103,238.9': '수출(중량)','5,387,753.0': '수출(금액)','64,664,166.5': '수입(중량)','175,454,117.0': '수입(금액)'}, inplace=True)
grape_export.head(2)

,년,월,수출(중량),수출(금액),수입(중량),수입(금액)
0,2016,01,"8,205.1","22,573.0","3,656,023.0","10,540,076.0"
1,2016,02,"1,137.4","3,914.0","4,606,327.0","15,021,662.0"


In [123]:
# 년 평균 데이터 drop(결측치)

grape_export[grape_export['월'] == '합'].index

Int64Index([12, 25, 38, 51], dtype='int64')

In [124]:
grape_export.drop([12, 25, 38, 51], inplace=True)
grape_export.reset_index(drop=True, inplace=True)

grape_export.head(2)

,년,월,수출(중량),수출(금액),수입(중량),수입(금액)
0,2016,01,"8,205.1","22,573.0","3,656,023.0","10,540,076.0"
1,2016,02,"1,137.4","3,914.0","4,606,327.0","15,021,662.0"


In [127]:
grape_export.replace('(.*),(.*)', r'\1\2', regex=True, inplace = True)

In [129]:
grape_export = grape_export.astype({'년':'int64','월':'int64', '수출(중량)':'float','수출(금액)':'float', '수입(중량)':'float','수입(금액)':'float'})
grape_export.dtypes 

년           int64
월           int64
수출(중량)    float64
수출(금액)    float64
수입(중량)    float64
수입(금액)    float64
dtype: object

In [130]:
# 2020년 11월 데이터 4일까지만 존재하여 값 수정

export_202011 = grape_export.iloc[58:,2:].values
export202011 = []
for i in export_202011:
    export202011.append((i/30)*4)
export202011

[array([  51186.38666667,  862239.46666667,  682659.37333333,
        2028986.26666667])]

In [131]:
grape_export.loc[59] = [2020, 11, 51186.38,  862239.46, 682659.37, 2028986.26]
grape_export.drop([58],inplace=True)
grape_export.reset_index(drop=True, inplace=True)

In [641]:
grape_export = grape_export.astype({'년':'int64','월':'int64'})
grape_export.shape 

(59, 6)

In [289]:
date = grape2[['date','년','월']]
date = date.astype({'년':'int64', '월':'int64'})
date.shape

(1770, 3)

In [308]:
grape_export2 = grape_export.copy()

In [309]:
A = [];B = [];C = [];D = []
for x in range(59):
    if (x == 1) | (x == 49):
        for i in range(29):
            A.append(grape_export2['수출(중량)'].iloc[x] / 29)
            B.append(grape_export2['수출(금액)'].iloc[x] / 29)
            C.append(grape_export2['수입(중량)'].iloc[x] / 29)
            D.append(grape_export2['수입(금액)'].iloc[x] / 29)
    elif x in [13, 25, 37]:
        for i in range(28):
            A.append(grape_export2['수출(중량)'].iloc[x] / 28)
            B.append(grape_export2['수출(금액)'].iloc[x] / 28)
            C.append(grape_export2['수입(중량)'].iloc[x] / 28)
            D.append(grape_export2['수입(금액)'].iloc[x] / 28)
    elif x in [3,5,8,10, 15,17,20,22, 27,29,32,34, 39,41,44,46, 51,53,56]:
        for i in range(30):
            A.append(grape_export2['수출(중량)'].iloc[x] / 30)
            B.append(grape_export2['수출(금액)'].iloc[x] / 30)
            C.append(grape_export2['수입(중량)'].iloc[x] / 30)
            D.append(grape_export2['수입(금액)'].iloc[x] / 30)
    elif x == 58:
        for i in range(5):
            A.append(grape_export2['수출(중량)'].iloc[x] / 30)
            B.append(grape_export2['수출(금액)'].iloc[x] / 30)
            C.append(grape_export2['수입(중량)'].iloc[x] / 30)
            D.append(grape_export2['수입(금액)'].iloc[x] / 30)
    else:
        for i in range(31):
            A.append(grape_export2['수출(중량)'].iloc[x] / 31)
            B.append(grape_export2['수출(금액)'].iloc[x] / 31)
            C.append(grape_export2['수입(중량)'].iloc[x] / 31)
            D.append(grape_export2['수입(금액)'].iloc[x] / 31)

In [310]:
grape_export2 = pd.DataFrame({'수출(중량)':A,
                    '수출(금액)':B,
                    '수입(중량)':C,
                    '수입(금액)':D})
grape_export2

,수출(중량),수출(금액),수입(중량),수입(금액)
0,264.680645,728.161290,117936.225806,340002.451613
1,264.680645,728.161290,117936.225806,340002.451613
2,264.680645,728.161290,117936.225806,340002.451613
3,264.680645,728.161290,117936.225806,340002.451613
4,264.680645,728.161290,117936.225806,340002.451613
...,...,...,...,...
1766,1706.212667,28741.315333,22755.312333,67632.875333
1767,1706.212667,28741.315333,22755.312333,67632.875333
1768,1706.212667,28741.315333,22755.312333,67632.875333
1769,1706.212667,28741.315333,22755.312333,67632.875333


### 재배면적(년단위) (공통-포도)

In [165]:
grape_product = pd.read_csv(grape_path+'재배면적_생산량.csv')
grape_product.rename(columns={'년도':'년', '포도:면적(ha)':'재배면적(ha)'},inplace=True)
grape_product.drop(columns=['생산량(톤)'],inplace=True)
grape_product

,년,재배면적(ha)
0,2016,14946
1,2017,13107
2,2018,12795
3,2019,12676
4,2020,13183


### 실업률1 - 경제활동인구(월단위, 평균) (공통-포도)

In [698]:
economy = pd.read_csv(raw_path+'실업률_경제활동인구.csv')
economy

,date,경제활동인구 (천명),실업률 (%)
0,2016.01,"26,631",3.7
1,2016.02,"26,925",4.9
2,2016.03,"27,133",4.2
3,2016.04,"27,398",3.9
4,2016.05,"27,617",3.6
...,...,...,...
58,2020.11,"28,208",3.4
59,2021.08,"28,346",2.6
60,2021.09,"28,439",2.7
61,2021.10,"28,528",2.8


In [699]:
economy.drop([59, 60,61,62], inplace=True)
economy.replace('(.*),(.*)', r'\1\2', regex=True, inplace = True)
economy = economy.astype({'경제활동인구 (천명)':'int64', 'date':'str'})
economy['경제활동인구'] = economy['경제활동인구 (천명)']*1000 # 1000명 단위 -> 1명 단위로 변환
economy.drop(columns=['경제활동인구 (천명)'], inplace=True)

In [700]:
economy2 = economy.copy()

economy2['년'] = economy2.date.str.split('.').str[0]
economy2['월'] = economy2.date.str.split('.').str[1]
economy2['월'].replace('1','10', inplace=True)
economy2 = economy2.astype({'년':'int64', '월':'int64'})
economy2.drop(columns=['date'], inplace=True)

In [702]:
economy2 = economy2[['년', '월', '경제활동인구','실업률 (%)']]

print(economy2.shape)
economy2.head(2)

(59, 4)


,년,월,경제활동인구,실업률 (%)
0,2016,1,26631000,3.7
1,2016,2,26925000,4.9


In [695]:
A = [];B = [];
for x in range(59):
    if (x == 1) | (x == 49):
        for i in range(29):
            A.append(economy2['경제활동인구'].iloc[x] / 29)
            B.append(economy2['실업률 (%)'].iloc[x] / 29)
    elif x in [13, 25, 37]:
        for i in range(28):
            A.append(economy2['경제활동인구'].iloc[x] / 29)
            B.append(economy2['실업률 (%)'].iloc[x] / 29)
    elif x in [3,5,8,10, 15,17,20,22, 27,29,32,34, 39,41,44,46, 51,53,56]:
        for i in range(30):
            A.append(economy2['경제활동인구'].iloc[x] / 29)
            B.append(economy2['실업률 (%)'].iloc[x] / 29)
    elif x == 58:
        for i in range(5):
            A.append(economy2['경제활동인구'].iloc[x] / 29)
            B.append(economy2['실업률 (%)'].iloc[x] / 29)
    else:
        for i in range(31):
            A.append(economy2['경제활동인구'].iloc[x] / 29)
            B.append(economy2['실업률 (%)'].iloc[x] / 29)

In [711]:
economy3 = pd.DataFrame({'경제활동인구':A,
                    '실업률 (%)':B})
economy3 = economy3.round(2)
economy3 = economy3.astype({'경제활동인구':'int64'})
economy3

,경제활동인구,실업률 (%)
0,918310,0.13
1,918310,0.13
2,918310,0.13
3,918310,0.13
4,918310,0.13
...,...,...
1766,972689,0.12
1767,972689,0.12
1768,972689,0.12
1769,972689,0.12


### 실업률2 - 최저시급(년단위) (공통-포도)

In [747]:
hourly_wage = pd.read_csv(raw_path+'최저시급.csv')
hourly_wage.rename(columns={'년도':'년'}, inplace=True)
hourly_wage

,년,최저시급
0,2016,6030
1,2017,6470
2,2018,7540
3,2019,8350
4,2020,8590


----

## 캠벨얼리 외부데이터 합치기 (grape2)
>거래량, 가격, 자동차용경유, 소비자 물가 지수 = (grape) <br>
'+' 날씨(weather2), 수출입(grape_export2), 재배면적(grape_product), 샤인머스켓 거래량(shihemuscat)), 실업률_경제활동인구(econmy3), 최저시급<br>
weather2 => 남원, 영천, 수원, 거창, 구미

최종 프레임 > grape!

In [753]:
grape = pd.concat([grape, product],axis = 1)
grape = pd.concat([grape, oil],axis = 1)

grape.drop(columns=[ '고급휘발유','보통휘발유','실내등유','구분'],inplace=True)
grape.drop([1770], inplace=True)

shinemuscat_product = shinemuscat.iloc[:,2:3]

grape2 = pd.merge(grape, weather2)
grape2.head(2)

grape2.date = pd.to_datetime(grape2.date)
# grape2 = pd.concat([grape2, pd.get_dummies(grape2['요일'])], axis = 1)
grape2['년'] = grape2['date'].dt.year 
grape2['월'] = grape2['date'].dt.month 

print(grape.shape)

grape2 = pd.concat([grape2, shinemuscat_product],axis = 1)

print(grape2.shape)

(1770, 6)
(1770, 13)


In [755]:
grape2.columns

Index(['date', '요일', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)', '소비자물가지수', '자동차용경유',
       '재배지역_평균기온', '재배지역_평균운량', '재배지역_일강수량', '재배지역_일교차', '년', '월',
       '샤인마스캇_거래량(kg)'],
      dtype='object')

In [756]:
# # 수출입 데이터 합치기
grape2 = pd.concat([grape2, grape_export2],axis = 1)

# # 재배면적 데이터 합치기
grape2 = pd.merge(grape2, grape_product, on=['년'])

# 최저시급 데이터 합치기
grape2 = pd.merge(grape2, hourly_wage, on = ['년'])

# 실업률 데이터 합치기
grape2 = pd.concat([grape2, economy3], axis = 1)

grape2.head(2)

,date,요일,캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,...,월,샤인마스캇_거래량(kg),수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),최저시급,경제활동인구,실업률 (%)
0,2016-01-01,금요일,0.0,0.0,95.232,1188.71,0.30,1.11,0.0,12.05,...,1.0,0.0,264.680645,728.16129,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
1,2016-01-02,토요일,880.0,2014.0,95.232,1187.91,3.61,1.31,0.0,13.11,...,1.0,0.0,264.680645,728.16129,117936.225806,340002.451613,14946.0,6030.0,918310,0.13


In [761]:
grape2.drop(columns=['년', '월'], inplace=True)
grape2.drop([1770], inplace=True)

In [762]:
grape2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1770 entries, 0 to 1769
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1770 non-null   datetime64[ns]
 1   요일             1770 non-null   object        
 2   캠벨얼리_거래량(kg)   1770 non-null   float64       
 3   캠벨얼리_가격(원/kg)  1770 non-null   float64       
 4   소비자물가지수        1770 non-null   float64       
 5   자동차용경유         1770 non-null   float64       
 6   재배지역_평균기온      1770 non-null   float64       
 7   재배지역_평균운량      1770 non-null   float64       
 8   재배지역_일강수량      1770 non-null   float64       
 9   재배지역_일교차       1770 non-null   float64       
 10  샤인마스캇_거래량(kg)  1770 non-null   float64       
 11  수출(중량)         1770 non-null   float64       
 12  수출(금액)         1770 non-null   float64       
 13  수입(중량)         1770 non-null   float64       
 14  수입(금액)         1770 non-null   float64       
 15  재배면적(ha)       1770 n

In [763]:
grape2

,date,요일,캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,샤인마스캇_거래량(kg),수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),최저시급,경제활동인구,실업률 (%)
0,2016-01-01,금요일,0.0,0.0,95.232,1188.71,0.30,1.11,0.00,12.05,0.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
1,2016-01-02,토요일,880.0,2014.0,95.232,1187.91,3.61,1.31,0.00,13.11,0.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
2,2016-01-03,일요일,0.0,0.0,95.232,1187.10,4.66,1.54,0.00,13.80,0.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
3,2016-01-04,월요일,2703.8,3885.0,95.232,1185.85,4.72,0.92,0.00,10.44,0.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
4,2016-01-05,화요일,8810.0,2853.0,95.232,1184.01,0.14,0.68,0.00,7.07,0.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,6030.0,918310,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,2020-10-31,토요일,34392.5,2920.0,100.180,1125.45,10.93,4.46,0.00,14.47,111721.4,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,8590.0,969517,0.13
1766,2020-11-01,일요일,0.0,0.0,100.090,1124.51,11.45,9.87,4.16,3.28,0.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,8590.0,972689,0.12
1767,2020-11-02,월요일,37043.4,3039.0,100.090,1123.70,10.95,4.74,0.12,13.15,88354.3,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,8590.0,972689,0.12
1768,2020-11-03,화요일,30158.5,3153.0,100.090,1122.34,6.08,2.56,0.41,8.60,84795.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,8590.0,972689,0.12


In [765]:
# grape2.to_csv(processed_path+'grape_실업률추가.csv', index = 0)

In [326]:
# weather2.to_csv(processed_path+'grape_weather.csv', index=0)

----

# 샤인머스켓 외부데이터 불러오기 및 전처리
**날씨 데이터 지역 비율 >>> <br>
구미_날씨(김천) = 71.8%, 영천_날씨 = 20.3%, 금산_날씨(영동) = 7.9%**

거래량, 가격, 자동차용경유, 소비자 물가 지수, 날씨(4개-평균기온, 평균온량, 일강수량, 일교차), 수출입(4개), 재배면적, 블루베리 거래량(워드클라우드), 실업률_경제활동인구, 최저시급

### 날씨 데이터 가져오기 및 전처리  
=> 구미, 영천, 금산

In [784]:
gumi = pd.read_csv(shinemuscat_path+'구미_날씨.csv', index_col=0)
yeongcheon = pd.read_csv(shinemuscat_path+'영천_날씨.csv', index_col=0)
geumsan = pd.read_csv(shinemuscat_path+'금산_날씨.csv', index_col=0)

In [785]:
# 상관계수가 높아서 drop -> 평균기온 & 일교차 대체 사용

gumi.drop(columns=['최고기온', '최저기온'],inplace=True)
geumsan.drop(columns=['최고기온', '최저기온'],inplace=True)
yeongcheon.drop(columns=['최고기온', '최저기온'],inplace=True)

In [786]:
# 컬럼 이름 바꿈

gumi.rename(columns={'평균기온':'구미_평균기온','평균운량':'구미_평균운량','일강수량':'구미_일강수량','일교차':'구미_일교차'}, inplace=True)
geumsan.rename(columns={'평균기온':'금산_평균기온','평균운량':'금산_평균운량','일강수량':'금산_일강수량','일교차':'금산_일교차'}, inplace=True)
yeongcheon.rename(columns={'평균기온':'영천_평균기온','평균운량':'영천_평균운량','일강수량':'영천_일강수량','일교차':'영천_일교차'}, inplace=True)

In [787]:
shinemuscat_weather = pd.merge(gumi, geumsan)
shinemuscat_weather = pd.merge(weather, yeongcheon)

print(shinemuscat_weather.shape)
shinemuscat_weather.head(2)

(1771, 25)


,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,금산_일강수량,금산_일교차,거창_평균기온,거창_평균운량,거창_일강수량,거창_일교차,영천_평균기온,영천_평균운량,영천_일강수량,영천_일교차
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,0.0,11.9,-1.1,0.0,0.0,16.5,1.3,0.0,0.0,15.4
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,0.0,13.8,1.4,0.0,0.0,18.3,2.0,0.0,0.0,16.4


In [788]:
# 구미_날씨(김천) = 71.8%, 영천_날씨 = 20.3%, 금산_날씨(영동) = 7.9%

shinemuscat_weather2 = shinemuscat_weather.copy()

shinemuscat_weather2['재배지역_평균기온'] = shinemuscat_weather2['구미_평균기온'] * (71.8/100) + shinemuscat_weather2['영천_평균기온'] * (20.3/100) + shinemuscat_weather2['금산_평균기온'] * (7.9/100)
shinemuscat_weather2['재배지역_평균운량'] = shinemuscat_weather2['구미_평균운량'] * (71.8/100) + shinemuscat_weather2['영천_평균운량'] * (20.3/100) + shinemuscat_weather2['금산_평균운량'] * (7.9/100)
shinemuscat_weather2['재배지역_일강수량'] = shinemuscat_weather2['구미_일강수량'] * (71.8/100) + shinemuscat_weather2['영천_일강수량'] * (20.3/100) + shinemuscat_weather2['금산_일강수량'] * (7.9/100)
shinemuscat_weather2['재배지역_일교차'] =  shinemuscat_weather2['구미_일교차'] * (71.8/100) + shinemuscat_weather2['영천_일교차'] * (20.3/100) + shinemuscat_weather2['금산_일교차'] * (7.9/100)

shinemuscat_weather2.head(2)

,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,거창_일강수량,거창_일교차,영천_평균기온,영천_평균운량,영천_일강수량,영천_일교차,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,0.0,16.5,1.3,0.0,0.0,15.4,0.4958,0.0,0.0,13.4003
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,0.0,18.3,2.0,0.0,0.0,16.4,2.2930,0.0,0.0,14.1842


In [789]:
# 날짜 칼럼 추가
date = df['date']
date.loc[1770] = '2020-11-05'

In [790]:
shinemuscat_weather2['date'] = date
shinemuscat_weather2.head(2)

,날짜,구미_평균기온,구미_평균운량,구미_일강수량,구미_일교차,남원_평균기온,남원_평균운량,남원_일강수량,남원_일교차,수원_평균기온,...,거창_일교차,영천_평균기온,영천_평균운량,영천_일강수량,영천_일교차,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,date
0,2016년 1월 1일,0.4,0.0,0.0,13.0,0.0,0.0,0.0,11.8,1.6,...,16.5,1.3,0.0,0.0,15.4,0.4958,0.0,0.0,13.4003,2016-01-01
1,2016년 1월 2일,2.1,0.0,0.0,13.6,3.5,0.0,0.0,14.2,5.8,...,18.3,2.0,0.0,0.0,16.4,2.2930,0.0,0.0,14.1842,2016-01-02


In [791]:
shinemuscat_weather2 = shinemuscat_weather2[['date','재배지역_평균기온', '재배지역_평균운량', '재배지역_일강수량', '재배지역_일교차']]
shinemuscat_weather2 = shinemuscat_weather2.round(2)

print(shinemuscat_weather2.shape)
display(shinemuscat_weather2.head(2))
display(shinemuscat_weather2.tail(2))

(1771, 5)


,date,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
0,2016-01-01,0.50,0.0,0.0,13.40
1,2016-01-02,2.29,0.0,0.0,14.18


,date,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
1769,2020-11-04,5.43,0.00,0.0,12.22
1770,2020-11-05,7.45,4.13,0.0,17.95


In [385]:
# shinemuscat_weather2.to_csv(processed_path+'shinemuscat_weather.csv', index=False)

### 수출입, 재배면적 데이터 합치기(캠벨얼리값과 동일)

In [833]:
shinemuscat_export2 = grape_export2.copy() # 수출입
shinemuscat_product2 = grape_product.copy() # 재배면적

### 블루베리 거래량 데이터 가져오기 및 전처리

In [519]:
shinemuscat_blueberry = pd.read_csv(shinemuscat_path+'샤인머스켓_블루베리.csv') # 블루베리
shinemuscat_blueberry.head(2)

,일자,거래물량 (톤),거래금액 (억원)
0,2013,0.014,0.0015
1,2014,0.240,0.0149


In [520]:
# 1톤 = 907.185 kg으로 변환
shinemuscat_blueberry['거래물량'] = round(shinemuscat_blueberry['거래물량 (톤)']*907.185, 2)

# 기존 데이터 drop
shinemuscat_blueberry.drop(columns=['거래물량 (톤)','거래금액 (억원)'], inplace = True)

In [522]:
shinemuscat_blueberry.rename(columns={'일자':'년','거래물량':'블루베리_거래물량'}, inplace=True)
shinemuscat_blueberry.dtypes

년              int64
블루베리_거래물량    float64
dtype: object

In [523]:
shinemuscat_blueberry.drop([0,1,2,9], inplace=True)
shinemuscat_blueberry

,년,블루베리_거래물량
3,2016,1660.15
4,2017,306.63
5,2018,7201.23
6,2019,1686.46
7,2020,3538.93
8,2021,4351.77


In [559]:
date = grape_export[['년', '월']]

shinemuscat_blueberry2 = pd.merge(date, shinemuscat_blueberry, on=['년'])

In [562]:
# 거래물량 년 -> 일 평균으로 바꿈

A = [];B = [];C = [];D = []
for x in range(59):
    if (x == 1) | (x == 49):
        for i in range(29):
            A.append(shinemuscat_blueberry2['블루베리_거래물량'].iloc[x] / 29)
    elif x in [13, 25, 37]:
        for i in range(28):
            A.append(shinemuscat_blueberry2['블루베리_거래물량'].iloc[x] / 28)
    elif x in [3,5,8,10, 15,17,20,22, 27,29,32,34, 39,41,44,46, 51,53,56]:
        for i in range(30):
            A.append(shinemuscat_blueberry2['블루베리_거래물량'].iloc[x] / 30)
    elif x == 58:
        for i in range(5):
            A.append(shinemuscat_blueberry2['블루베리_거래물량'].iloc[x] / 30)
    else:
        for i in range(31):
            A.append(shinemuscat_blueberry2['블루베리_거래물량'].iloc[x] / 31)

In [569]:
shinemuscat_blueberry3 = pd.DataFrame({'블루베리_거래물량':A})
shinemuscat_blueberry3 = shinemuscat_blueberry3.round(2)
shinemuscat_blueberry3

,블루베리_거래물량
0,53.55
1,53.55
2,53.55
3,53.55
4,53.55
...,...
1766,117.96
1767,117.96
1768,117.96
1769,117.96


### 샤인머스켓 외부데이터 합치기
> 거래량, 가격, 자동차용경유, 소비자 물가 지수, 날씨(5개), 수출입, 재배면적, 블루베리 거래량<br>
**금산_날씨 = 7.9%, '영천_날씨 = 20.3%, 구미_날씨 = 71.8%**

### 소비자 물가지수, 자동차용경유, 날씨

In [845]:
shinemuscat2 = shinemuscat.copy()

In [846]:
shinemuscat2 = pd.concat([shinemuscat2, product],axis = 1)
shinemuscat2 = pd.concat([shinemuscat2, oil],axis = 1)

shinemuscat2.drop(columns=[ '고급휘발유','보통휘발유','실내등유','구분'],inplace=True)

shinemuscat2.drop([1770], inplace=True)

In [847]:
shinemuscat2 = pd.merge(shinemuscat2, shinemuscat_weather2, on = ['date'])
shinemuscat2.head(2)

,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차
0,2016-01-01,금요일,0.0,0.0,95.232,1188.71,0.50,0.0,0.0,13.40
1,2016-01-02,토요일,0.0,0.0,95.232,1187.91,2.29,0.0,0.0,14.18


### 요일, 월 분할

In [848]:
shinemuscat2.date = pd.to_datetime(shinemuscat2.date)
# shinemuscat = pd.concat([shinemuscat, pd.get_dummies(shinemuscat['요일'])], axis = 1)
shinemuscat2['년'] = shinemuscat2['date'].dt.year 
shinemuscat2['월'] = shinemuscat2['date'].dt.month 

In [849]:
print(shinemuscat2.shape)

(1770, 12)


In [851]:
# 수출입 데이터 합치기
shinemuscat2 = pd.concat([shinemuscat2, shinemuscat_export2],axis = 1)

# 재배면적 데이터 합치기
shinemuscat2 = pd.merge(shinemuscat2, shinemuscat_product2, on=['년'])

# 블루베리 데이터 합치기
shinemuscat2 = pd.concat([shinemuscat2, shinemuscat_blueberry3], axis = 1 )

# 최저시급 데이터 합치기
shinemuscat2 = pd.merge(shinemuscat2, hourly_wage, on = ['년'])

# 실업률 데이터 합치기
shinemuscat2 = pd.concat([shinemuscat2, economy3], axis = 1)

shinemuscat2

,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,...,월,수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),블루베리_거래물량,최저시급,경제활동인구,실업률 (%)
0,2016-01-01,금요일,0.0,0.0,95.232,1188.71,0.50,0.00,0.00,13.40,...,1.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,53.55,6030.0,918310,0.13
1,2016-01-02,토요일,0.0,0.0,95.232,1187.91,2.29,0.00,0.00,14.18,...,1.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,53.55,6030.0,918310,0.13
2,2016-01-03,일요일,0.0,0.0,95.232,1187.10,3.59,0.00,0.00,16.56,...,1.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,53.55,6030.0,918310,0.13
3,2016-01-04,월요일,0.0,0.0,95.232,1185.85,6.17,0.00,0.00,10.06,...,1.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,53.55,6030.0,918310,0.13
4,2016-01-05,화요일,0.0,0.0,95.232,1184.01,1.53,0.00,0.00,6.04,...,1.0,264.680645,728.161290,117936.225806,340002.451613,14946.0,53.55,6030.0,918310,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,2020-11-01,일요일,0.0,0.0,100.090,1124.51,11.45,9.91,2.53,3.41,...,11.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590.0,972689,0.12
1767,2020-11-02,월요일,88354.3,10205.0,100.090,1123.70,11.40,5.50,0.00,13.69,...,11.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590.0,972689,0.12
1768,2020-11-03,화요일,84795.0,10322.0,100.090,1122.34,6.68,1.78,0.01,8.72,...,11.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590.0,972689,0.12
1769,2020-11-04,수요일,74970.5,10178.0,100.090,1121.22,5.43,0.00,0.00,12.22,...,11.0,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590.0,972689,0.12


In [852]:
shinemuscat2.drop(columns=['년', '월'], inplace=True)
shinemuscat2.drop([1770], inplace=True)

In [857]:
shinemuscat2.rename(columns={'블루베리_거래물량':'블루베리_거래물량(kg)'}, inplace=True)
shinemuscat2 = shinemuscat2.astype({'최저시급':'int64'})

display(shinemuscat2.head(2))
display(shinemuscat2.tail(2))
print(shinemuscat2.shape)

,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),블루베리_거래물량(kg),최저시급,경제활동인구,실업률 (%)
0,2016-01-01,금요일,0.0,0.0,95.232,1188.71,0.50,0.0,0.0,13.40,264.680645,728.16129,117936.225806,340002.451613,14946.0,53.55,6030,918310,0.13
1,2016-01-02,토요일,0.0,0.0,95.232,1187.91,2.29,0.0,0.0,14.18,264.680645,728.16129,117936.225806,340002.451613,14946.0,53.55,6030,918310,0.13


,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),블루베리_거래물량(kg),최저시급,경제활동인구,실업률 (%)
1768,2020-11-03,화요일,84795.0,10322.0,100.09,1122.34,6.68,1.78,0.01,8.72,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590,972689,0.12
1769,2020-11-04,수요일,74970.5,10178.0,100.09,1121.22,5.43,0.00,0.00,12.22,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590,972689,0.12


(1770, 19)


In [858]:
shinemuscat2.to_csv(processed_path+'shinemuscat_실업률추가.csv', index = False)

In [859]:
shinemuscat2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1770 entries, 0 to 1769
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1770 non-null   datetime64[ns]
 1   요일              1770 non-null   object        
 2   샤인마스캇_거래량(kg)   1770 non-null   float64       
 3   샤인마스캇_가격(원/kg)  1770 non-null   float64       
 4   소비자물가지수         1770 non-null   float64       
 5   자동차용경유          1770 non-null   float64       
 6   재배지역_평균기온       1770 non-null   float64       
 7   재배지역_평균운량       1770 non-null   float64       
 8   재배지역_일강수량       1770 non-null   float64       
 9   재배지역_일교차        1770 non-null   float64       
 10  수출(중량)          1770 non-null   float64       
 11  수출(금액)          1770 non-null   float64       
 12  수입(중량)          1770 non-null   float64       
 13  수입(금액)          1770 non-null   float64       
 14  재배면적(ha)        1770 non-null   float64       
 15  블루베리

In [860]:
grape2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1770 entries, 0 to 1769
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1770 non-null   datetime64[ns]
 1   요일             1770 non-null   object        
 2   캠벨얼리_거래량(kg)   1770 non-null   float64       
 3   캠벨얼리_가격(원/kg)  1770 non-null   float64       
 4   소비자물가지수        1770 non-null   float64       
 5   자동차용경유         1770 non-null   float64       
 6   재배지역_평균기온      1770 non-null   float64       
 7   재배지역_평균운량      1770 non-null   float64       
 8   재배지역_일강수량      1770 non-null   float64       
 9   재배지역_일교차       1770 non-null   float64       
 10  샤인마스캇_거래량(kg)  1770 non-null   float64       
 11  수출(중량)         1770 non-null   float64       
 12  수출(금액)         1770 non-null   float64       
 13  수입(중량)         1770 non-null   float64       
 14  수입(금액)         1770 non-null   float64       
 15  재배면적(ha)       1770 n